# 프로토콜(Protocol)
## 객체의 설계도
## 델리게이션(Delegation)

### 프로토콜 타입으로 선언된 값을 사용한다는 것은,
- #### 여기에 할당된 객체가 구체적으로 어떤 기능을 갖추고 있는지는 상관 없다는 뜻이기도 하다
- #### 그저 단순히 할당된 객체를 사용하여 프로토콜에 정의된 프로퍼티나 메소드를 호출하겠다는 의미가 된다

### 코코아 터치 프레임워크에서는 이러한 프로토콜 타입의 특성을 이용하여 델리게이션이라는 기능을 구현한다

### 델리게이션(Delegation)은 델리게이트 패턴과 연관되는 아주 중요한 개념
- #### 간략하게 설명하자면 특정 기능을 다른 객체에 위임하고, 그에 따라 필요한 시점에서 메소드의 호출만 받는 패턴이라고 할 수 있다

### 이해를 돕기 위한 예시
- 자동차, 오토바이, 기차 등에서 공통으로 사용되는 연료 펌프를 전문으로 만드는 회사가 있다
    - 이 회사에서 만들어낸 연료펌프는 특별히 신경 쓰지 않아도 알아서 잘 굴러가다가 연료가 부족해지면 이를 연료펌프가 장착된 시스템에 알려주는 기능이 있다고 가정
    - 이 알림을 받은 자동차, 기차 등의 시스템은 각자의 방법으로 모자란 연료를 보충하게 된다
        - 또, 연료 보충 과정에서 충분히 연료가 채워지면 이를 시스템에 알려 연료 보충을 중단하도록 알려주기도 한다고 가정
    - 이때 연료가 부족해지는 시점이나 연료가 가득 차는 시점을 연료펌프가 장착된 시스템이 점검하는 것이 아니라 연료펌프에 위임
        - #### 연료펌프가 알아서 스스로 알려주도록 하는 것이 바로 델리게이션이라고 할 수 있다

### 위의 예시를 코드로 표현
- #### 먼저 두 개의 객체가 필요하다

### 먼저 연료의 양에 따라 필요한 알람을 전달하게 될 FuelPumpDelegate 프로토콜

```
protocol FuelPumpDelegate {
    func lackFuel()
    func fullFuel()
}
```

- #### 이 프로토콜은 두 개의 메소드로 이루어져 있다
    - #### 하나는 연료가 부족할 때 호출되는 메소드
    - #### 또 다른 하나는 연료가 가득 찼을 때 호출되는 메소드
        - #### 이 메소드들을 자동차, 오토바이 등 각 객체에서 나름대로 구현하게 된다
- #### 연료펌프는 이 객체들의 메소드만 호출하여 연료를 보충하거나 보충을 중단한다


### 연료펌프 클래스

```
class FuelPump {
    var maxGage: Double = 100.0
    var delegate: FuelPumpDelegate? = nil
    
    
    var fuelGage: Double {
        didSet {
            if oldValue < 10 {
                // 연료가 부족해지면 델리게이트의 lackFule 메소드를 호출한다.
                self.delegate?.lackFuel()
            } else if oldValue == self.maxGage {
                // 연료가 가득차면 델리게이트의 fullFuel 메소드를 호출한다.
                self.delegat?.fullFuel()
            }
        }
    }
    
    
    init(fuelGage: Double = 0) {
        self.fuelGage = fuelGage
    }
    // 연료펌프를 가동한다.
    func startPump() {
        while (true) {
            if (self.fuelGage > 0) {
                self.jetFuel()
            } else {
                break
            }
        }
    }
    
    
    // 연료를 엔진에 분사한다. 분사할 때마다 연료 게이지의 눈금은 내려간다.
    func jetFuel() {
        self.fuelGage -= 1
    }
}
```

- #### 이 클래스는 FuelPumpDelegate 프로토콜을 구현한 객체의 정보를 delegate 프로퍼티에 저장해두었다가, 필요한 시점에 프로토콜 메소드를 호출하는 대상으로 사용한다
- #### 또한, 연료 눈금을 의미하는 프로퍼티 fuelGage에 대한 프로퍼티 옵저버를 작성하여 연료 눈금이 변화할 때마다 적정 수치를 검사하고 10 미만으로 떨어지면 델리게이트 프로퍼티에 저장된 객체에 lackFuel( )메소드를, 연료가 가득 차면 fullFuel( )메소드를 각각 호출한다

### 이때 delegate 프로퍼티에 저장되는 객체는 FuelPumpDelegate 프로토콜 타입으로 선언된다
- #### delegate 프로퍼티는 선언된 타입으로 인해, 실제 그 객체가 어떤 타입이든지 관계없이 FuelPumpDelegate 프로토콜에 정의된 lackFuel()과 fullFuel() 메소드만을 사용할 수 있다
- #### 할당된 인스턴스가 나머지 프로퍼티나 메소드들도 분명 구현하고 있겠지만, 여기에서는 그 정보를 알 필요는 없다
- #### 그저 필요한 시점에서 lackFuel( )과 fullFuel( ) 메소드들을 호출할 수 있으면 그것으로 충분한 것이다

### 연료펌프 클래스를 이동수단에 장착해 보는 예시
- #### 이동수단이 되는 클래스는 반드시 FuelPumpDelegate 프로토콜을 구현해야 한다
- #### 그래야 연료펌프 클래스의 delegate 프로퍼티에 자신을 할당할 수 있는 타입이 될 수 있다

```
class Car: FuelPumpDelegate {
    var fuelPump = FuelPump(fuelGage: 100)
    
    init() {
        self.fuelPump.delegate = self
    }
    
    
    // fuelPump가 호출하는 메소드입니다.
    func lackFuel() {
        // 연료를 보충한다.
    }
    
    
    // fuelPump가 호출하는 메소드입니다.
    func fullFuel() {
        // 연료 보충을 중단한ㄷ.
    }
    
    
    // 자동차에 시동을 건다
    func start() {
        fuelPump.startPump()
    }
}
```

### 작성된 클래스는 Car라는 이름의 클래스
- #### fuelPump라는 프로퍼티에 앞에서 작성한 연료펌프 클래스의 인스턴스를 할당한다
- #### 초기화 구문을 통해 클래스가 만들어질 때 연료펌프에 연료를 100으로 채우고, 연료펌프의 델리게이트 프로퍼티를 자신으로 설정한다
- #### 이제 Car 클래스를 누군가 인스턴스로 생성하여 start 메소드를 호출하면 연료펌프 역시 작동되면서 연료가 부족해지는 시점이 오면 delegate 객체를 대상으로 lackFuel() 메소드를 호출한다
- #### delegate 프로퍼티에는 Car의 인스턴스가 할당되어 있으므로 Car 클래스에서 작성한 lackFuel() 메소드가 실행된다

### 약간 복잡해 보이는 구성이지만, 
- #### 델리게이트 참조를 통해 메소드를 호출할 인스턴스 객체를 전달받고, 이 인스턴스 객체가 구현하고 있는 프로토콜에 선언된 메소드를 호출하는 것이 델리게이션이라고 할 수 있다

### 그렇다면 왜 프로토콜일까?
- #### 클래스를 이용하는 경우에도 부모 클래스를 상속받은 자식 클래스의 인스턴스들은 모두 부모 클래스 타입으로 정의된 변수나 상수에 할당할 수 있었다
- #### 그리고 부모 클래스 타입으로 선언된 변수나 상수에 할당된 자식 클래스의 인스턴스들은 모두 자식 클래스에서 구현한 프로퍼티나 메소드들을 봉인 당해야 했다
- #### 이를 꺼내어 사용하려면 타입 캐스팅 과정을 거쳐야 했지만 어쨋든 각 클래스의 종류와 관계없이 같은 클래스를 부모로 둔 자식 클래스들은 모두 부모 클래스 타입으로 선언된 변수/상수에 할당되므로 이를 사용하면 될 일이다

### 그런데도 프로토콜을 사용하여 이처럼 델리게이션을 구현하는 것은,
- #### 클래스가 단일 상속만을 지원하기 때문이다
    - #### 하나의 부모 클래스를 상속받고 나면 더는 다른 클래스를 상속받을 수 없으므로 기능을 덧붙이기에는 제한적이다

### 이를 극복하기 위해 구현 개수에 제한이 없는 프로토콜을 이용하여 필요한 기능 단위별 객체를 작성하는 것이다

### iOS 앱이 동작하는 방식 대다수가 델리게이트 패턴으로 이루어져 있고, 델리게이트 패턴을 이루는 핵심이 바로 프로토콜이다
- #### 그런 만큼 프로토콜이 사용되는 방식과 프로토콜을 이용하여 델리게이션을 구현하는 원리에 대해 자세히 알아둘 필요가 있다